In [48]:
import holoviews as hv
import hvplot.pandas
import intake
import panel as pn

hv.streams.PlotSize.scale = 0.2

pn.extension('tabulator', design='material', template='material')

In [52]:
cat = intake.open_anaconda_catalog('examples')['uswtdb']

df = pn.state.as_cached('windturbines', cat.wind_turbines.read)

df['x'], df['y'] = hv.element.tiles.lon_lat_to_easting_northing(df.xlong, df.ylat)

df.head()


,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,retrofit,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,x,y
0,3072695.0,<NA>,<NA>,5143.0,52161.0,CA,Kern County,6029.0,251 Wind,1987.0,...,0.0,NaN,2.0,3.0,5/8/2018,Digital Globe,-118.364410,35.077435,-1.317627e+07,4.174409e+06
1,3072661.0,<NA>,<NA>,5149.0,52161.0,CA,Kern County,6029.0,251 Wind,1987.0,...,0.0,NaN,2.0,3.0,5/8/2018,Digital Globe,-118.363762,35.077908,-1.317619e+07,4.174474e+06
2,3072704.0,<NA>,<NA>,5146.0,52161.0,CA,Kern County,6029.0,251 Wind,1987.0,...,0.0,NaN,2.0,3.0,5/8/2018,Digital Globe,-118.364197,35.077644,-1.317624e+07,4.174438e+06
3,3063269.0,19-028130,2016-WTE-5934-OE,NaN,NaN,IA,Story County,19169.0,30 MW Iowa DG Portfolio,2017.0,...,0.0,NaN,3.0,3.0,7/23/2017,Digital Globe,-93.632835,41.882477,-1.042316e+07,5.143391e+06
4,3057059.0,19-027954,2016-WTE-9485-OE,NaN,NaN,IA,Story County,19169.0,30 MW Iowa DG Portfolio,2017.0,...,0.0,NaN,3.0,3.0,5/13/2017,Digital Globe,-93.623009,41.881470,-1.042207e+07,5.143241e+06


In [21]:
ls = hv.link_selections.instance()

plot = df.hvplot.points('x', 'y', rasterize=True, cmap='viridis', cnorm='eq_hist', xaxis=None, yaxis=None, tiles='CartoDark', responsive=True, min_height=500)

plot_ls = ls(plot)

plot_ls

:DynamicMap   []
   :Overlay
      .Tiles.I      :Tiles   [x,y]
      .Image.I      :Image   [x,y]   (x_y Count)
      .Image.II     :Image   [x,y]   (x_y Count)
      .Rectangles.I :Rectangles   [x0,y0,x1,y1]
      .Path.I       :Path   [x,y]

In [36]:
df_sel = ls.selection_param(df)

dfi_sel = hvplot.bind(lambda df: df, df_sel).interactive()[['t_state', 't_county', 'p_year', 'p_name', 'p_cap']]

table = pn.widgets.Tabulator(dfi_sel, pagination='remote', page_size=10, sizing_mode='stretch_width')

table

Tabulator(design=<class 'panel.theme.materi..., page_size=10, pagination='remote', sizing_mode='stretch_width', value=      t_state  ...)

In [46]:
groupby = pn.widgets.RadioButtonGroup(options=['p_year', 't_state', 'eia_id'])
topn = pn.widgets.IntSlider(start=5, end=50, name='Top N')

agg_bar_plot = dfi_sel.groupby(groupby)[['p_cap']].sum().sort_values('p_cap', ascending=False).iloc[:topn].hvplot.bar(rot=45)

agg_bar_plot

In [ ]:
pn.Column(
    plot_ls, 
    pn.Row(
        table, agg_bar_plot
    )
).servable("Table")